In [1]:
import pandas as pd
import pickle
import csv

In [2]:
%%time
import pickle

with open('work_item_rest_data.pickle', 'rb') as raw_data:
    all_wits = pickle.load(raw_data)


CPU times: user 3.55 s, sys: 789 ms, total: 4.34 s
Wall time: 4.38 s


In [3]:
%%time
import sys
csv.field_size_limit(sys.maxsize)
with open("work_item_comments.tsv") as fd:
    rd = csv.reader(fd, delimiter="\t", quoting=csv.QUOTE_NONE)
    for row in rd:
        id = int(str(row[0]).replace('\ufeff',''))
        comments = row[1]
        if id in all_wits:
            if 'System.Comments' in all_wits[id].fields:
                all_wits[id].fields['System.Comments'] = ' '.join([all_wits[id].fields['System.Comments'], comments])
            else:
                all_wits[id].fields['System.Comments'] = comments


CPU times: user 5.49 s, sys: 237 ms, total: 5.73 s
Wall time: 5.73 s


In [4]:
%%time
from tqdm import tqdm_notebook as tqdm
from TextCleaning import TextCleanUtils as cleaner

ids = [id for id in all_wits]

def get_area():
    print('area')
    x = []
    for id in tqdm(ids):
        x.append(all_wits[id].fields['System.AreaPath'])
    return x

def get_type():
    print('type')
    x = []
    for id in tqdm(ids):
        x.append(all_wits[id].fields['System.WorkItemType'])
    return x

def get_project():
    print('project')
    x = []
    for id in tqdm(ids):
        x.append(all_wits[id].fields['System.TeamProject'])
    return x

def get_title():
    print('title')
    x = []
    for id in tqdm(ids):
        x.append(cleaner.clean_text(all_wits[id].fields['System.Title']))
    return x

description_fields = [
    'System.Description',
    'Microsoft.VSTS.TCM.ReproSteps',
    'Microsoft.DevDiv.WalkThroughDescription'
]

def get_description():
    print('description')
    x = []
    for id in tqdm(ids):
        x.append(cleaner.clean_text(' '.join([all_wits[id].fields[f] if f in all_wits[id].fields else '' for f in description_fields])))
    return x

def get_tags():
    print('tags')
    x = []
    for id in tqdm(ids):
        x.append(cleaner.clean_text(all_wits[id].fields['System.Tags'] if 'System.Tags' in all_wits[id].fields else ''))
    return x

def get_comments():
    print('comments')
    x = []
    for id in tqdm(ids):
        x.append(cleaner.clean_text(all_wits[id].fields['System.Comments'] if 'System.Comments' in all_wits[id].fields else ''))
    return x

d = {
    'id': [id for id in all_wits],
    'area': get_area(),
    'type': get_type(),
    'project': get_project(),
    'title': get_title(),
    'description': get_description(),
    'comments' : get_comments(),
    'tags': get_tags()
}

raw_df = pd.DataFrame(d)

area



type



project



title



description



comments



tags



CPU times: user 23min 27s, sys: 18 s, total: 23min 45s
Wall time: 1h 21min 45s


In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [6]:
raw_df.sample(20)

id  \
428215  1282918   
110634  457282    
431885  1290729   
209310  703469    
218051  796360    
85764   371292    
295100  986466    
271007  931806    
406527  1237558   
46393   207899    
190968  664380    
300889  999025    
328790  1063789   
286870  966950    
256367  912484    
381950  1183747   
331609  1071485   
279363  950145    
67136   280701    
118268  478232    

                                                                                                        area  \
428215  VSOnline\VSTS\RM and Deployment\Deployment-Core                                                        
110634  AppInsights\Dev Experience\API\Power BI                                                                
431885  VSOnline\VSTS\Agile\WIT X                                                                              
209310  VSOnline\VSTS\Agile\WIT X                                                                              
218051  VSOnline\z_Deprecated - Do Not Use\Continuous Integration\CP-Old-Do-Not-Use                            
85764   VSOnline\VSTS\NC Services Platform\Cloud Admin and Tools                                               
295100  VSOnline\VSTS\Version Control\VC Server\SSH Old                                                        
271007  TechnicalContent\CustomerFeedback\Verbatim                                                             
406527  VSOnline\VSTS\CI and Web Platform\CIX                                                                  
46393   VSOnline\z_Deprecated - Do Not Use                                                                     
190968  VSChina\OpenPublishing\MigrationAndXamarin                                                             
300889  CSI\APEX - Zhiliang's Team                                                                             
328790  TechnicalContent\SW\BusApps AppPlat Intelligence\BAPI Group\Application Platform\Microsoft Flow\Flow   
286870  VSOnline\VSTS\Marketplace                                                                              
256367  CSI\APEX - Jonathan's Team\Validation                                                                  
381950  TechnicalContent\Data and Analytics\SQL\SQL Server                                                     
331609  TechnicalContent\Azure Core\Azure Identity\Identity ITPros                                             
279363  VSOnline\VSTS\Version Control\VC Desktop\Team Explorer                                                 
67136   VSOnline\VSTS\CI and Web Platform\Build                                                                
118268  VSOnline\VSTS\CI and Web Platform\Build                                                                

              type           project  \
428215  User Story  VSOnline           
110634  Bug         AppInsights        
431885  User Story  VSOnline           
209310  User Story  VSOnline           
218051  Bug         VSOnline           
85764   Bug         VSOnline           
295100  Bug         VSOnline           
271007  Bug         TechnicalContent   
406527  User Story  VSOnline           
46393   User Story  VSOnline           
190968  User Story  VSChina            
300889  Feature     CSI                
328790  Bug         TechnicalContent   
286870  User Story  VSOnline           
256367  Bug         CSI                
381950  User Story  TechnicalContent   
331609  Bug         TechnicalContent   
279363  Bug         VSOnline           
67136   Bug         VSOnline           
118268  Bug         VSOnline           

                                                                                          title  \
428215  azure web app measure performance improvments                                             
110634  drill user last day user last day box go report                                           
431885  board backcompat                                                                          
209310  add batch updat

In [7]:
with open('clean_wit_dataframe_no_stemming.pickle', 'wb') as handle:
    pickle.dump(raw_df, handle)